In [1]:
import pandas as pd
import sys
sys.path.insert(0, "/media/diskD/EPFL/Fall 2016/ADA/Project/GMR_ADA_Project/EmotionAnalysis")
from DataSchemaExtractionParsing import *
from DataPreProcessing import *
from SentSemanticModule import *
from SentTweetModule import *
from SentSyntacticModule import *
import math

/home/meryem/miniconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


### Loading English Data 

In [6]:
english_tweets = pd.read_csv("/home/meryem/Dropbox/meryem/algorithms/EmotionRecognition/Notebooks/WholeTranscriptDatasetWithoutContractions.txt",encoding ="utf-8")
english_tweets.head()

/home/meryem/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,Actor,MovieId,SeriesName,Subtitle
0,0,0,Sheldon,NaN,BBT,So if a photon is directed through a plane wi...
1,1,1,Leonard,NaN,BBT,"Agreed, what is your point?"
2,2,2,Sheldon,NaN,BBT,"There’s no point, I just think it is a good i..."
3,3,3,Leonard,NaN,BBT,Excuse me?
4,4,4,Receptionist,NaN,BBT,Hang on.


In [3]:
len(english_tweets)

10000

In [4]:
english_tweets.columns

Index([u'id', u'userId', u'createdAt', u'text', u'longitude', u'latitude',
       u'placeId', u'inReplyTo', u'source', u'truncated', u'placeLatitude',
       u'placeLongitude', u'sourceName', u'sourceUrl', u'userName',
       u'screenName', u'followersCount', u'friendsCount', u'statusesCount',
       u'userLocation', u'swiss', u'canton', u'language'],
      dtype='object')

In [12]:
english_tweets['text'] = english_tweets['Subtitle']

In [21]:
english_tweets['Subtitle'].iloc[877]

u' 120?'

### Replacing Special Categories

In [13]:
replaced_categories = handle_special_categories(english_tweets[0:1000])
len(replaced_categories)

/media/diskD/EPFL/Fall 2016/ADA/Project/GMR_ADA_Project/EmotionAnalysis/DataPreProcessing.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  sample_tweets['text'] = tweets_list


1000

In [14]:
replaced_categories['text'].iloc[0]

u' So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. If it is unobserved it will, however, if it is observed after it is left the plane but before it hits its target, it will not have gone through both slits.'

### Replacing contractions (needed for more accurate tokenization)

In [15]:
tweets_no_contractions = replace_contractions(replaced_categories)

In [16]:
tweets_no_contractions['text'][0]

u' So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. If it is unobserved it will, however, if it is observed after it is left the plane but before it hits its target, it will not have gone through both slits.'

### Tokenization of Tweets into words

In [17]:
tokenized_list = bag_of_word_representation(tweets_no_contractions)

In [18]:
tokenized_list[0]

[u'So',
 u'if',
 u'a',
 u'photon',
 u'is',
 u'directed',
 u'through',
 u'a',
 u'plane',
 u'with',
 u'two',
 u'slits',
 u'in',
 u'it',
 u'and',
 u'either',
 u'slit',
 u'is',
 u'observed',
 u'it',
 u'will',
 u'not',
 u'go',
 u'through',
 u'both',
 u'slits',
 u'If',
 u'it',
 u'is',
 u'unobserved',
 u'it',
 u'will',
 u'however',
 u'if',
 u'it',
 u'is',
 u'observed',
 u'after',
 u'it',
 u'is',
 u'left',
 u'the',
 u'plane',
 u'but',
 u'before',
 u'it',
 u'hits',
 u'its',
 u'target',
 u'it',
 u'will',
 u'not',
 u'have',
 u'gone',
 u'through',
 u'both',
 u'slits']

### Part of Speech Tagging:

In [19]:
tagged_tweets = pos_tagging(tokenized_list)
tagged_tweets[0]

[(u'So', 'RB'),
 (u'if', 'IN'),
 (u'a', 'DT'),
 (u'photon', 'NN'),
 (u'is', 'VBZ'),
 (u'directed', 'VBN'),
 (u'through', 'IN'),
 (u'a', 'DT'),
 (u'plane', 'NN'),
 (u'with', 'IN'),
 (u'two', 'CD'),
 (u'slits', 'NNS'),
 (u'in', 'IN'),
 (u'it', 'PRP'),
 (u'and', 'CC'),
 (u'either', 'DT'),
 (u'slit', 'NN'),
 (u'is', 'VBZ'),
 (u'observed', 'VBN'),
 (u'it', 'PRP'),
 (u'will', 'MD'),
 (u'not', 'RB'),
 (u'go', 'VB'),
 (u'through', 'IN'),
 (u'both', 'DT'),
 (u'slits', 'NNS'),
 (u'If', 'IN'),
 (u'it', 'PRP'),
 (u'is', 'VBZ'),
 (u'unobserved', 'JJ'),
 (u'it', 'PRP'),
 (u'will', 'MD'),
 (u'however', 'RB'),
 (u'if', 'IN'),
 (u'it', 'PRP'),
 (u'is', 'VBZ'),
 (u'observed', 'VBN'),
 (u'after', 'IN'),
 (u'it', 'PRP'),
 (u'is', 'VBZ'),
 (u'left', 'VBN'),
 (u'the', 'DT'),
 (u'plane', 'NN'),
 (u'but', 'CC'),
 (u'before', 'IN'),
 (u'it', 'PRP'),
 (u'hits', 'VBZ'),
 (u'its', 'PRP$'),
 (u'target', 'NN'),
 (u'it', 'PRP'),
 (u'will', 'MD'),
 (u'not', 'RB'),
 (u'have', 'VB'),
 (u'gone', 'VBN'),
 (u'through', 'I

In [14]:
unannotated_tweets_df = pd.read_csv('Results/nava representation.csv')
unannotated_tweets_df['Tokenized Lemmatized'] = lemmatizer_raw(tagged_tweets)
unannotated_tweets_df.to_csv('Results/Unannotated_Representation.csv')

### Dependency Parser

In [ ]:
# STANFORD VERSION : More accurate but is too slow:
#import os
#os.environ["STANFORD_MODELS"] = "/home/meryem/Downloads/stanford-parser-full-2016-10-31"
#os.envir
#on["STANFORD_PARSER"] = "/home/meryem/Downloads/stanford-parser-full-2016-10-31"
#from nltk.parse.stanford import StanfordDependencyParser
#dep_parser=StanfordDependencyParser(model_path="/home/meryem/Downloads/stanford-parser-full-2016-10-31/edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")
#dependency_trees = []
#for tweet in tweets_no_contractions['text']:
    #result = dep_parser.raw_parse("No rest is detrimental")
    #dep = result.next()
    #dependency_trees.append(list(dep.triples()))

In [10]:
# TEMPORARY SOLUTION FOR DEPENDENCY PARSING:
# to install Run the following:
# pip install -U spacy
# python -m spacy.en.download all # for ENGLISH
# python -m spacy.de.download all # for Deutch
nlp = spacy.load('en')

In [26]:
docs = []
# Joining text:
tweets_text = []
for i in range(0, len(tokenized_list)):
    space = u" "
    tweets_text.append(space.join(tokenized_list[i]))
tweets_text[0].encode("utf-8")
for i in range(0, len(tweets_text)):
    doc = nlp(tweets_text[i])
    docs.append(doc)

In [28]:
nlp(u'')

#tweets_text[877]

In [29]:
new_samples = []
for sample in docs:
    new_samples_sub = []
    for word in sample:
        new_samples_sub.append((unicode(word),word.pos_))
    new_samples.append(new_samples_sub)

### Application of Syntactic Rules:

In [31]:
new_samples,triple_dependencies = apply_syntactic_rules(docs,new_samples)

In [32]:
i = 0
print "\n<<<< Original tweet text >>>\n"
print tweets_text[i]
print "\n<<<< Syntactic dependencies >>>\n"
print triple_dependencies[i]
print "\n<<<< Tweet after applying syntactic Rules >>>\n"
new_tweet = []
for (word,pos) in new_samples[i]:
    new_tweet.append(word)
print new_tweet


<<<< Original tweet text >>>

So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits If it is unobserved it will however if it is observed after it is left the plane but before it hits its target it will not have gone through both slits

<<<< Syntactic dependencies >>>

[(So, u'advmod', observed), (if, u'mark', directed), (a, u'det', photon), (photon, u'nsubjpass', directed), (is, u'auxpass', directed), (directed, u'advcl', observed), (through, u'prep', directed), (a, u'det', plane), (plane, u'pobj', through), (with, u'prep', directed), (two, u'nummod', slits), (slits, u'pobj', with), (in, u'prep', slits), (it, u'pobj', in), (and, u'cc', directed), (either, u'advmod', slit), (slit, u'conj', directed), (is, u'auxpass', observed), (observed, u'ROOT', observed), (it, u'nsubj', go), (will, u'aux', go), (not, u'neg', go), (go, u'ccomp', observed), (through, u'prep', go), (both, u'det', slits), (slits, u'pobj', through),

In [17]:
print tagged_tweets[0]
print "\n"
print new_samples[0]

[(u'Still', 'RB'), (u'the', 'DT'), (u'best', 'JJS'), (u'coffee', 'NN'), (u'in', 'IN'), (u'town', 'NN'), (u'at', 'IN'), (u'La', 'NNP'), (u'Stanza', 'NNP')]


[(u'the', u'DET'), (u'best', u'ADJ'), (u'in', u'ADP'), (u'town', u'NOUN'), (u'at', u'ADP'), (u'La', u'PROPN'), (u'Stanza', u'PROPN')]


### Named Entity Tagging:

In [18]:
tweet_without_ne = remove_named_entities(new_samples)

### Normalizing POS tag

In [19]:
normalized_tags = normalize_pos_tags_words(tweet_without_ne)
normalized_tags[1]

[(u'ready', u'ADJ')]

### Removal of Punctuation and Stop words and Converting to Lower Case and Removal of Other special categories: url, number, username:

In [20]:
tagged_tweets_without = eliminate_stop_words_punct(normalized_tags)

### Lemmatization:

In [21]:
lemmatized_tweets = lemmatizer(tagged_tweets_without)

lemmatized_tweets_untag = lemmatizer_untagged(tagged_tweets_without)
lemmatized_tweets[0]

[(u'best', u'ADJ'), (u'town', 'n'), (u'la', u'PROPN'), (u'stanza', u'PROPN')]

### Keeping only NAVA words

In [22]:
nava_tweets = keep_only_nava_words(lemmatized_tweets)

In [24]:
nava_tweets[0]

[u'best', u'town']

In [25]:
nava_tweets_df = pd.DataFrame()
nava_tweets_df['Nava Tweets'] = nava_tweets
nava_tweets_df.to_csv('Results/nava representation.csv',index=False)

### Extracting NRC Lexicon:

In [24]:
# EXECUTE THIS TO DIRECTLY LOAD THE PROCESSED LEXICON
lexicon_df = pd.read_csv('NRCLexicon/lexicon_nrc.csv',encoding='utf-8')

In [18]:
lexicon = extractLexicon()
word_set = list(set(lexicon['Word']))
word_emotional_vectors = []
for word in word_set:
    word_emotional_vectors.append((word,list(lexicon[lexicon['Word']==word]['Score'])))
word_emotional_vectors_dict = dict(word_emotional_vectors)

In [19]:
lexicon.head(1)
word_emotional_vectors_dict['happy']

['0', '1', '0', '0', '1', '0', '1', '0', '0', '1']

In [20]:
emo_dict = {
    0: 'Anger',
    1: 'Anticipation',
    2: 'Disgust',
    3: 'Fear',
    4: 'Joy',
    5: 'Negative',
    6: 'Positive',
    7: 'Sadness',
    8: 'Surprise',
    9: 'Trust',
    10: 'Neutral'
}
emotion_ids = []
for word in word_emotional_vectors_dict.keys():
    for i in range(0,len(word_emotional_vectors_dict[word])):
        if word_emotional_vectors_dict[word][i] == '1':
            emotion_ids.append((word,i))

In [21]:
emotion_repres_words_list = [] 
for i in range(0,10):
    emotion_repres_words_list_sub = []
    for (word,emotion) in emotion_ids:
        if emotion == i:
            emotion_repres_words_list_sub.append(word)
    emotion_repres_words_list.append(emotion_repres_words_list_sub)
len(emotion_repres_words_list[5])

3324

In [22]:
lexicon_df = pd.DataFrame()
lexicon_df[0] = emotion_repres_words_list[0]
for i in range(1,10):
    df = pd.DataFrame()
    df[i] = emotion_repres_words_list[i]
    lexicon_df= pd.concat([lexicon_df,df],ignore_index=True,axis=1)

In [23]:
lexicon_df.to_csv('NRCLexicon/lexicon_nrc.csv',index=False)

In [8]:
import pandas as pd
lexicon_df = pd.read_csv('NRCLexicon/lexicon_nrc.csv')
lexicon_df.columns

Index([u'0', u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9'], dtype='object')

In [25]:
unique_lexicon = make_unique_lexicon(lexicon_df)

### Calculating Semantic Similarity using PMI:

In [26]:
flatten_list = [word for sublist in lemmatized_tweets for (word, tag) in sublist]

NameError: name 'lemmatized_tweets' is not defined

In [ ]:
clean_pmi_dict = calculate_pmi(flatten_list,unique_lexicon)

In [ ]:
emotion_pmi_based = compute_matrix_sentences_list(nava_tweets,lexicon_df, clean_pmi_dict)

In [ ]:
emo_dict = {
    0: 'Anger',
    1: 'Anticipation',
    2: 'Disgust',
    3: 'Fear',
    4: 'Joy',
    #5: 'Negative',
    #6: 'Positive',
    5: 'Sadness',
    6: 'Surprise',
    7: 'Trust',
    8: 'Neutral'
}
sent_dict = {
    0: "Positive",
    1: "Negative",
    2: "Neutral"
}

In [ ]:
# Emotion Recognition
sentence_vectors_pmi = compute_sentence_emotion_vectors(emotion_pmi_based)

emotionalities = compute_emotionalities(sentence_vectors_pmi)


# Sentiment Analysis
sentence_vectors_sent_pmi = compute_sentence_sentiment_vectors(emotion_pmi_based)

sentiments = compute_sentiments(sentence_vectors_sent_pmi)

### Visualizing the results

In [ ]:
english_tweets['Affective Feature Representation'] = lemmatized_tweets
english_tweets['Emotion Ids'] = sentence_vectors_pmi
emotions = []
senti = []
for i in range(0,len(emotionalities)):
    emotions.append(emo_dict[emotionalities[i]])
    senti.append(sent_dict[sentiments[i]])
english_tweets['Emotionalities'] = emotions
english_tweets['Sentiments'] = senti

In [ ]:
english_tweets_col = english_tweets[['text','Affective Feature Representation','Emotion Ids','Emotionalities','Sentiments']]
english_tweets_col.to_csv('PMI_Lexicon_ResultsSampleEnglishData.csv',encoding ="utf-8",index=False)

In [ ]:
%matplotlib inline
from collections import Counter
frequency = Counter(english_tweets_col['Emotionalities'])
df = pd.DataFrame.from_dict(frequency, orient='index')
df.plot(kind='bar')

In [ ]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
a=np.random.random(9)
cs=cm.Set1(np.arange(9)/9.)
f=plt.figure()
ax=f.add_subplot(111, aspect='equal')
patches, texts = plt.pie(df, colors=cs, startangle=90)
labels = df.index
p=plt.pie(df, colors=cs, labels = labels)
plt.axis('equal')
plt.tight_layout()
plt.title("PMI- Rule Based Fine-grained Emotion Distribution")
plt.show()

### Visualizing Sentiment Results

In [ ]:
%matplotlib inline
from collections import Counter
frequency = Counter(english_tweets_col['Sentiments'])
df = pd.DataFrame.from_dict(frequency, orient='index')
df.plot(kind='bar')

In [ ]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
a=np.random.random(3)
cs=cm.Set1(np.arange(3)/3.)
f=plt.figure()
ax=f.add_subplot(111, aspect='equal')
patches, texts = plt.pie(df, colors=cs, startangle=90)
labels = df.index
p=plt.pie(df, colors=cs, labels = labels)
plt.axis('equal')
plt.tight_layout()
plt.title("PMI- Rule Based Sentiment Emotion Distribution")
plt.show()

# II. Same approach using word2vec similarity

### 1. Training Word2Vec Model on the whole tweets:

In [ ]:
raw_tokenized_lemma = lemmatizer_raw(tagged_tweets)
len(raw_tokenized_lemma)
raw_tokenized_lemma[0]

In [ ]:
model = train_word2Vec_model(300, 40, 4, 10, 1e-3 , raw_tokenized_lemma, "geo_tweets_word2vec_model"): 

In [ ]:
model.similarity('happy','so')

In [ ]:
emotion_word2vec_based = compute_matrix_sentences_list_word2vec(lemmatized_tweets,lexicon_df,model)

In [ ]:
emotion_word2vec_based

In [ ]:
import math

# Emotion Recognition
sentence_vectors_word2vec = compute_sentence_emotion_vectors(emotion_word2vec_based)

emotionalities = compute_emotionalities(sentence_vectors_word2vec)



# Sentiment Analysis
sentence_vectors_sent_word2vec = compute_sentence_sentiment_vectors(emotion_word2vec_based)

sentiments = compute_sentiments(sentence_vectors_sent_word2vec)

### Storing Results

In [ ]:
english_tweets['Affective Feature Representation'] = lemmatized_tweets
english_tweets['Emotion Ids'] = sentence_vectors_pmi
emotions = []
senti = []
for i in range(0,len(emotionalities)):
    emotions.append(emo_dict[emotionalities[i]])
    senti.append(sent_dict[sentiments[i]])
english_tweets['Emotionalities'] = emotions
english_tweets['Sentiments'] = senti

In [ ]:
english_tweets_col = english_tweets[['text','Affective Feature Representation','Emotion Ids','Emotionalities','Sentiments']]
english_tweets_col.to_csv('Word2Vec_Lexicon_ResultsSampleEnglishData.csv',encoding ="utf-8",index=False)

### Visualizing the distribution

In [ ]:
%matplotlib inline
from collections import Counter
frequency = Counter(english_tweets_col['Emotionalities'])
df = pd.DataFrame.from_dict(frequency, orient='index')
df.plot(kind='bar')

In [ ]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
a=np.random.random(9)
cs=cm.Set1(np.arange(9)/9.)
f=plt.figure()
ax=f.add_subplot(111, aspect='equal')
patches, texts = plt.pie(df, colors=cs, startangle=90)
labels = df.index
p=plt.pie(df, colors=cs, labels = labels)
plt.axis('equal')
plt.tight_layout()
plt.title("PMI- Rule Based Fine-grained Emotion Distribution")
plt.show()

### Visualizing Sentiment Results

In [ ]:
%matplotlib inline
from collections import Counter
frequency = Counter(english_tweets_col['Sentiments'])
df = pd.DataFrame.from_dict(frequency, orient='index')
df.plot(kind='bar')

In [ ]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
a=np.random.random(3)
cs=cm.Set1(np.arange(3)/3.)
f=plt.figure()
ax=f.add_subplot(111, aspect='equal')
patches, texts = plt.pie(df, colors=cs, startangle=90)
labels = df.index
p=plt.pie(df, colors=cs, labels = labels)
plt.axis('equal')
plt.tight_layout()
plt.title("PMI- Rule Based Sentiment Emotion Distribution")
plt.show()